In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [3]:
from we import get_we, initiate_model

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from scipy.stats import f_oneway

In [6]:
from sklearn.metrics import mean_absolute_error, accuracy_score

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [9]:
labels = [m['label'] for m in models]

In [10]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [11]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [12]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'POS'

# Verbs

In [13]:
pos = ['VERB']

Split each model into train and test:

In [14]:
X_verb_train = []
y_verb_train = []

X_verb_test = []
y_verb_test = []

In [15]:
for we in we_with_features:
    xtr, xtst, ytr, ytst = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='VERB',
                                           split=True,
                                           balance=True)
    X_verb_train.append(xtr)
    X_verb_test.append(xtst)
    
    y_verb_train.append(ytr)
    y_verb_test.append(ytst)

In [16]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['Train size'] = [len(x) for x in X_verb_train]
dataset_sizes['Test size'] = [len(x) for x in X_verb_test]
dataset_sizes

,Train size,Test size
flau_small_c,5099,1275
flau_base_u,6070,1518
flau_base_c,5099,1275
flau_large_c,5099,1275
cam_base,3115,779
xlm_large,1388,348
xlm_base,1388,348
bert_base_u,1966,492
distilbert_base,926,232
bert_base_c,926,232


## Non-independent dims

ANOVA test with p-value < 0.01:

In [17]:
anova_dims = [get_anova_dims(X_verb_train[i], y_verb_train[i]) for i in range(len(models))]

In [18]:
mi_dims = [get_mi_dims(X_verb_train[i], y_verb_train[i]) for i in range(len(models))]

In [19]:
non_ind_dims = [list(set(anova_dims[i]).intersection(mi_dims[i])) for i in range(len(models))]

Stats about the number of dimensions for each model:

In [20]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [21]:
non_ind_df['ANOVA'] = [len(x) for x in anova_dims]
non_ind_df['MI'] = [len(x) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x) for x in non_ind_dims]

In [22]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,442,457,412
flau_base_u,579,766,579
flau_base_c,523,575,418
flau_large_c,832,893,757
cam_base,415,656,357
xlm_large,542,709,427
xlm_base,62,438,43
bert_base_u,382,458,236
distilbert_base,292,482,216
bert_base_c,17,721,17


In [23]:
dims = {}

In [24]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = X_verb_train[i].columns
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [25]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [26]:
lr_res = [lr(X_verb_train[i], y_verb_train[i]) for i in range(len(models))]

Train Perceptron 10 times and get average weights:

In [27]:
perceptron_res = [perceptron(X_verb_train[i], y_verb_train[i]) for i in range(len(models))]

Compute correlation to the gender vector:

In [28]:
corr_res = [correlation(X_verb_train[i], y_verb_train[i]) for i in range(len(models))]

In [29]:
for i in range(len(models)):
    for alpha in alphas:
        num_imp_dims = len(X_verb_test[i].columns)*alpha//100
        lr_dims = [str(x[0]) for x in lr_res[i][:num_imp_dims]]
        perc_dims = [str(x[0]) for x in perceptron_res[i][:num_imp_dims]]
        corr_dims = [str(x[0]) for x in corr_res[i][:num_imp_dims]]
        dims[labels[i]][f'LR{alpha}'] = lr_dims
        dims[labels[i]][f'Perc{alpha}'] = perc_dims
        dims[labels[i]][f'Corr{alpha}'] = corr_dims
        dims[labels[i]][f'All imp dims{alpha}'] = list(set(lr_dims).intersection(perc_dims).intersection(corr_dims))

## Compute medians

In [30]:
medians = {}

In [31]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = {}
        dim_list = dims[labels[i]][dim_group]
        # Median of dimensions where feature vector is equal to 0
        median_0 = X_verb_train[i][y_verb_train[i] == 0][dim_list].median()
        # Median of dimensions where feature vector is equal to 1
        median_1 = X_verb_train[i][y_verb_train[i] == 1][dim_list].median()
        
        medians[labels[i]][dim_group]['0'] = median_0
        medians[labels[i]][dim_group]['1'] = median_1


In [32]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group])

In [33]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,512,768,768,1024,768,1024,768,768,768,768
ANOVA,442,579,523,832,415,542,62,382,292,17
MI,457,766,575,893,656,709,438,458,482,721
All non ind,412,579,418,757,357,427,43,236,216,17
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,0,0,0,2,0,0,1,0,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [34]:
y_preds = {}

In [ ]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        dim_list = dims[labels[i]][dim_group]
        mae0 = X_verb_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['0'], x), axis=1)
        mae1 = X_verb_test[i][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group]['1'], x), axis=1)
        # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
        y_preds[labels[i]][dim_group] = (mae0 > mae1).apply(int)
    

## Compute accuracies

In [ ]:
accs = {}

In [ ]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_true = y_verb_test[i]
        y_pred = y_preds[labels[i]][dim_group]
        if any(y_pred):
            acc = accuracy_score(y_true, y_pred)
        else:
            acc = 0
        accs[labels[i]][dim_group] = acc

In [ ]:
accs_df = pd.DataFrame(accs)
accs_df

We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [ ]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [ ]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

In [ ]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

In [ ]:
if 'pos_verb.pickle' not in os.listdir('../Data/best_results/'):
    best_res = {}
    for label in labels:
        best_res[label] = {}
        best_res[label]['best_dim_set'] = []
        best_res[label]['best_dims'] = []
        best_res[label]['accs'] = []
        best_res[label]['gains'] = []
        best_res[label]['medians_0'] = []
        best_res[label]['medians_1'] = []


else:
    with open('../Data/best_results/pos_verb.pickle', 'rb') as f:
        best_res = pickle.load(f)

In [ ]:
for i in range(len(models)):
    best_res[labels[i]]['best_dim_set'].append(gains_df.idxmax()[i])
    best_res[labels[i]]['best_dims'].append(dims[labels[i]][gains_df.idxmax()[i]])
    best_res[labels[i]]['accs'].append(accs_df.loc[accs_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['gains'].append(gains_df.loc[gains_df.idxmax()[i], labels[i]])
    best_res[labels[i]]['medians_0'].append(medians[labels[i]][accs_df.idxmax()[i]]['0'])
    best_res[labels[i]]['medians_1'].append(medians[labels[i]][accs_df.idxmax()[i]]['1'])

In [ ]:
with open('../Data/best_results/pos_verb.pickle', 'wb') as f:
    pickle.dump(best_res, f)

# Conclusion